## 2019-2020 CitiBike Analysis

In order to use 24 months of CitiBike data in Tableau Public, it has to be reduced to a reasonable size. This notebook was used to remove unused columns and clean and prepare data for Tableau including adding months to each instance for plotting.

In [1]:
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
citi_data_4 = Path("output/202002-citibike-tripdata.csv")

# Read each month's Data File and store into Pandas DataFrames
citi_data = pd.read_csv(citi_data_4)

Import each month's csv and reduce to necessary columns along with finding ride totals and average trips for each station.

In [2]:
citi_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1404,2020-02-01 00:00:05.9460,2020-02-01 00:23:30.7240,316,Fulton St & William St,40.709560,-74.006536,481,S 3 St & Bedford Ave,40.712605,-73.962644,28874,Customer,1995,1
1,1301,2020-02-01 00:00:06.2230,2020-02-01 00:21:48.0580,237,E 11 St & 2 Ave,40.730473,-73.986724,539,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,32588,Subscriber,1991,1
2,474,2020-02-01 00:00:15.7210,2020-02-01 00:08:10.3440,528,2 Ave & E 31 St,40.742909,-73.977061,3785,W 42 St & 6 Ave,40.754920,-73.984550,41013,Subscriber,1994,1
3,487,2020-02-01 00:00:21.0520,2020-02-01 00:08:28.7520,380,W 4 St & 7 Ave S,40.734011,-74.002939,3263,Cooper Square & Astor Pl,40.729515,-73.990753,27581,Subscriber,1973,2
4,619,2020-02-01 00:00:27.4000,2020-02-01 00:10:47.0640,472,E 32 St & Park Ave,40.745712,-73.981948,237,E 11 St & 2 Ave,40.730473,-73.986724,29062,Subscriber,1994,1


In [3]:
# Limit to just the columns that are being used
citi_data_small = citi_data.drop(["starttime", "stoptime", "end station id", \
                "end station name", "end station latitude", \
                "end station longitude", "bikeid", "birth year", "usertype", "gender"], axis=1) 

citi_data_small.head()


,tripduration,start station id,start station name,start station latitude,start station longitude
0,1404,316,Fulton St & William St,40.709560,-74.006536
1,1301,237,E 11 St & 2 Ave,40.730473,-73.986724
2,474,528,2 Ave & E 31 St,40.742909,-73.977061
3,487,380,W 4 St & 7 Ave S,40.734011,-74.002939
4,619,472,E 32 St & Park Ave,40.745712,-73.981948


In [4]:
# Check for NaNs
citi_data_small.dropna()

,tripduration,start station id,start station name,start station latitude,start station longitude
0,1404,316,Fulton St & William St,40.709560,-74.006536
1,1301,237,E 11 St & 2 Ave,40.730473,-73.986724
2,474,528,2 Ave & E 31 St,40.742909,-73.977061
3,487,380,W 4 St & 7 Ave S,40.734011,-74.002939
4,619,472,E 32 St & Park Ave,40.745712,-73.981948
...,...,...,...,...,...
1146825,203,3737,Stanton St & Norfolk St,40.720747,-73.986274
1146826,357,546,E 30 St & Park Ave S,40.744449,-73.983035
1146827,169,3043,Lewis Ave & Decatur St,40.681460,-73.934903
1146828,738,3255,8 Ave & W 31 St,40.750585,-73.994685


In [5]:
#Build smaller db from rows using groupby

station_latitudes = citi_data_small.groupby(['start station id'])["start station latitude"].unique().tolist()
station_longitudes = citi_data_small.groupby(['start station id'])["start station longitude"].unique().tolist()
station_names = citi_data_small.groupby(['start station id'])["start station name"].unique().tolist()
station_ids = citi_data_small.groupby(['start station id'])['start station id'].unique().tolist()
ride_counts = citi_data_small.groupby(['start station id'])['start station id'].size()
minutes_per_station = citi_data_small.groupby(['start station id'])['tripduration'].sum()

year_data = pd.DataFrame({"date": 20200201, "station_id": station_ids, "station_name": station_names,\
                          "station_lat": station_latitudes, "station_long": station_longitudes, \
                          "ride_counts": ride_counts, "trip_minutes": minutes_per_station})
year_data.head()

In [12]:
#Reformatting for dates, int, removing []

year_data["station_id"] = year_data["station_id"].str[0]
year_data["station_name"] = year_data["station_name"].str[0]
year_data["station_lat"] = year_data["station_lat"].str[0]
year_data["station_long"] = year_data["station_long"].str[0]

year_data["station_id"] = year_data["station_id"].astype(int)
year_data["date"] = pd.to_datetime(year_data["date"], format='%Y%m%d')

year_data.head()


,date,station_id,station_name,station_lat,station_long,ride_counts,trip_minutes
start station id,,,,,,,
72,2020-02-01,72,W 52 St & 11 Ave,40.767272,-73.993929,2323,2085284
79,2020-02-01,79,Franklin St & W Broadway,40.719116,-74.006667,1319,924443
82,2020-02-01,82,St James Pl & Pearl St,40.711174,-74.000165,798,550458
83,2020-02-01,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,1159,768384
116,2020-02-01,116,W 17 St & 8 Ave,40.741776,-74.001497,4446,2718221


In [13]:
# Check dtypes
year_data.dtypes

date            datetime64[ns]
station_id               int64
station_name            object
station_lat            float64
station_long           float64
ride_counts              int64
trip_minutes             int64
dtype: object

In [14]:
year_data.to_csv('202002.csv')